# Figures in supplementary

Håkon Wiik Ånes (hakon.w.anes@ntnu.no), 2022-11-29, Norwegian University of Science and Technology (NTNU)

Some figures in the supplementary are generated in the `figures_in_paper` notebook.

In [ ]:
# Switch to interactive Matplotlib backend (e.g. qt5) for control point determination
%matplotlib qt5

from datetime import date
import importlib_metadata
import os

import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import scipy.ndimage as ndi
import numpy as np
import skimage.exposure as ske
import skimage.color as skc
import skimage.transform as skt


# Interactive plotting OFF
plt.ioff()

# Other Matplotlib parameters
scalebar_kwds = dict(
    units="um",
    dimension="si-length",
    location="lower left",
    pad=0.2,
    sep=3,
    border_pad=0.5,
    box_alpha=0.6,
    fixed_value=15,
)
plt.rcParams.update({"font.size": 11, "figure.dpi": 300})
savefig_kw = dict(bbox_inches="tight", pad_inches=0.01, transparent=True)

print("Run date: ", date.today())
print("\nSoftware versions\n-----------------")
for pkg in ["matplotlib", "numpy", "scikit-image"]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

In [ ]:
step_size = dict(ebsd=100, bse=1 / 39.2e-3)
scale_ebsd_bse = step_size["ebsd"] / step_size["bse"]

In [ ]:
ebsd_pad = {
    "0s": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "175c": {1: [(100, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(200, 100), (100, 0)]},
    "300c": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(0, 0), (0, 0)]},
    "325c": {1: [(0, 0), (0, 0)], 2: [(0, 0), (0, 0)], 3: [(100, 0), (0, 0)]},
}

In [ ]:
sample = "325c"
dset_no = 3
dir_article = "/home/hakon/phd/artikler_abstract/202204_aanes_p_texture_almn_alloy_actamat/latex/fig"
dir_data = os.path.join("/home/hakon/phd/data/p/prover", sample, str(dset_no))
dir_bse = os.path.join(dir_data, "bse")
dir_mtex = os.path.join(dir_data, "mtex")
dir_imreg = os.path.join(dir_data, "imreg")
dir_partdet = os.path.join(dir_data, "partdet")

Read backscatter electron (BSE) image from region of interest (ROI) of electron backscatter diffraction (EBSD) map

In [ ]:
bse = plt.imread(os.path.join(dir_imreg, "reg_bse_no_padding.png"))
bse = skc.rgb2gray(bse[..., :3])

# Cut to ROI
mask1 = bse != 0
mask1 = ndi.binary_fill_holes(mask1)
y1, x1 = np.where(mask1)
ymin1, ymax1 = np.min(y1), np.max(y1)
xmin1, xmax1 = np.min(x1), np.max(x1)
bse = bse[ymin1:ymax1, xmin1:xmax1]

# Blank out black inside ROI
mask2 = bse != 0
mask2 = ndi.binary_fill_holes(mask2)
bse[~mask2] = np.nan

Read BSE image with particles detected using *ImageJ*.
The image is cropped to the same ROI as the above image (quite a lot of wrangling to enable use of a common scalebar).

In [ ]:
bse_part = plt.imread(os.path.join(dir_partdet, "bse_particles_roi.png"))

# Upscale to enable use of EBSD ROI mask from image registration notebook
bse0 = plt.imread(os.path.join(dir_bse, "4500x_cropped2_fused_cropped_cropped.png"))
bse_part = skt.rescale(bse_part, tuple(np.array(bse0.shape) / np.array(bse_part.shape[:2])) + (1,))

# Read mask
mask3 = np.load(os.path.join(dir_imreg, "mask_ebsd_correct.npy"))
mask3 = skt.rescale(mask3, scale_ebsd_bse)

# Pad particle BSE image or crop mask and EBSD ROI, depending on which have the
# greater shape
mask3_bse_shape_diff = np.array(mask3.shape) - np.array(bse_part.shape[:2])
if mask3_bse_shape_diff[0] > 0:
    # Add rows to the end of the BSE image
    bse_part2 = np.pad(bse_part, [(0, mask3_bse_shape_diff[0]), (0, 0), (0, 0)])
else:
    # Add rows to the end of the rescaled mask and EBSD intensity map
    mask3 = mask3[:bse_part.shape[0]]
    mask3 = np.pad(mask3, [(0, -mask3_bse_shape_diff[0]), (0, 0)])
    bse_part2 = bse_part.copy()
if mask3_bse_shape_diff[1] > 0:
    # Add columns to the end of the BSE image
    bse_part2 = np.pad(bse_part2, [(0, 0), (0, mask3_bse_shape_diff[1]), (0, 0)])
else:
    # Add columns to the end of the rescaled mask and EBSD intensity map
    mask3 = mask3[:, :bse_part.shape[1]]
    mask3 = np.pad(mask3, [(0, 0), (0, -mask3_bse_shape_diff[1])])

# Apply mask to BSE particle image
bse_part2[~mask3] = 0

# Remove black borders
mask4 = skc.rgb2gray(bse_part2[..., :3]) != 0
mask4 = ndi.binary_fill_holes(mask4)
y4, x4 = np.where(mask4)
ymin4, ymax4 = np.min(y4), np.max(y4)
xmin4, xmax4 = np.min(x4), np.max(x4)
bse_part3 = bse_part2[ymin4:ymax4, xmin4:xmax4]

# Crop to fit above BSE image
bse_part3 = bse_part3[:bse.shape[0], :bse.shape[1]]

Pad both BSE images to fit the EBSD ROI exactly

In [ ]:
pad = np.array(ebsd_pad[sample][dset_no]) / scale_ebsd_bse
pad = np.round(pad).astype(int)

bse2 = np.pad(bse, pad, constant_values=np.nan)
bse_part4 = np.pad(bse_part3, list(pad) + [(0, 0)])

Read inverse pole figure (IPF) map with orientations colored according to the lattice vector parallel to the rolling direction (RD) upwards

In [ ]:
ideal_map = plt.imread(os.path.join(dir_mtex, "maps_grains_ideal_particles.png"))
ideal_map = ideal_map[3:-3, 3:-3]

Combine images in a figure with a common scalebar

In [ ]:
figsize = (9 * bse.shape[1] / max(bse.shape), 3 * bse.shape[0] / max(bse.shape))
fig, axes = plt.subplots(ncols=3, figsize=figsize)
axes[0].imshow(bse2, cmap="gray")
axes[1].imshow(bse_part4)
axes[2].imshow(ideal_map)
axes[0].add_artist(ScaleBar(dx=step_size["bse"] * 1e-3, **scalebar_kwds))
for ax in axes:
    ax.axis("off")
fig.tight_layout(pad=0.3)
fig.savefig(os.path.join(dir_article, "maps", f"maps_{sample}_{dset_no}.png"), **savefig_kw)